# **Easy Financial Visualizations with Plotly**

In [51]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get Koç Holding data using the clean method
ticker = yf.Ticker("KCHOL.IS")
df = ticker.history(period="3mo")

"""
print("Data columns:", df.columns.tolist())
print("Data shape:", df.shape)
print("\nFirst few rows:")
print(df.head())
"""

# Create OHLC-V chart with volume below
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    row_width=[0.2, 0.8]  # 70% for OHLC, 30% for volume
)

# Add OHLC candlestick chart (top panel)
fig.add_trace(
    go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Price',
        increasing_line_color='green',
        decreasing_line_color='red'
    ),
    row=1, col=1
)

# Add volume bars (bottom panel)
fig.add_trace(
    go.Bar(
        x=df.index,
        y=df['Volume'],
        name='Volume',
        marker_color='lightblue',
        opacity=0.7
    ),
    row=2, col=1
)

# Update layout
fig.update_layout(
    title='KCHOL.IS - OHLC with Volume',
    width=1000,
    height=700,
    showlegend=False,
    template='plotly_white',
    xaxis_rangeslider_visible=False
)

# Configure Y-axes on the RIGHT side
fig.update_yaxes(
    title_text="Price (TRY)",
    side="right",
    row=1, col=1
)

fig.update_yaxes(
    title_text="Volume",
    side="right",
    row=2, col=1
)

# Configure X-axis
fig.update_xaxes(
    title_text="Date",
    row=2, col=1
)

# Hide weekends for cleaner look
fig.update_xaxes(
    rangebreaks=[{'bounds': ['sat', 'mon']}]
)

# Show the chart
fig.show()

In [49]:
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Get data for both stocks
ticker1 = yf.Ticker("KCHOL.IS")
ticker2 = yf.Ticker("ASELS.IS")

df1 = ticker1.history(period="3mo")
df2 = ticker2.history(period="3mo")

print("KCHOL.IS columns:", df1.columns.tolist())
print("ASELS.IS columns:", df2.columns.tolist())

# Create figure with secondary Y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add first stock (left Y-axis)
fig.add_trace(
    go.Scatter(
        x=df1.index,
        y=df1['Close'],
        name='KCHOL.IS',
        line=dict(color='blue', width=2),
        hovertemplate='KCHOL: ₺%{y:.2f}<extra></extra>'
    ),
    secondary_y=False  # Left Y-axis
)

# Add second stock (right Y-axis)
fig.add_trace(
    go.Scatter(
        x=df2.index,
        y=df2['Close'],
        name='ASELS.IS',
        line=dict(color='red', width=2),
        hovertemplate='ASELS: ₺%{y:.2f}<extra></extra>'
    ),
    secondary_y=True  # Right Y-axis
)

# Update layout
fig.update_layout(
    title='KCHOL.IS vs ASELS.IS - Dual Y-Axis Price Comparison',
    xaxis_title='Date',
    width=1000,
    height=600,
    showlegend=True,
    hovermode='x unified',
    template='plotly_white'
)

# Set Y-axes titles and styles
fig.update_yaxes(
    title_text="KCHOL.IS Price (TRY)",
    secondary_y=False,  # Left axis
    side="left",
    showgrid=True,
    gridcolor='lightgray'
)

fig.update_yaxes(
    title_text="ASELS.IS Price (TRY)",
    secondary_y=True,   # Right axis
    side="right",
    showgrid=False  # No grid for right axis to avoid clutter
)

# Hide weekends
fig.update_xaxes(
    rangebreaks=[{'bounds': ['sat', 'mon']}],
    rangeslider_visible=False
)

fig.show()

KCHOL.IS columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']
ASELS.IS columns: ['Open', 'High', 'Low', 'Close', 'Volume', 'Dividends', 'Stock Splits']


In [ ]:
!pip install yfinance plotly pandas numpy --quiet

In [50]:
import yfinance as yf
import pandas as pd
import plotly.express as px
import numpy as np

# --- Define stocks ---
stocks = ["KCHOL.IS", "ASELS.IS", "THYAO.IS"]

# --- Download historical data ---
data = yf.download(stocks, start="2025-06-01", end="2025-09-01", progress=False)

# --- Reshape to long format ---
df = data.stack(level=1).reset_index()  # columns: Date, Ticker, Open, High, Low, Close, Volume
df = df.rename(columns={'Date':'date', 'Close':'price', 'Volume':'volume'})

# --- Add bubble size (scaled volume) ---
df['size'] = df['volume'] / df['volume'].max() * 100  # scale for visibility

# --- Add log price for Y-axis ---
df['log_price'] = np.log(df['price'])

# --- Convert date to string for animation frames ---
df['date_str'] = df['date'].dt.strftime('%Y-%m-%d')

# --- Motion chart with log Y-axis ---
fig = px.scatter(
    df,
    x='date',             # time
    y='log_price',        # log stock price for Y-axis
    size='size',          # bubble size = volume
    color='Ticker',       # stock identity
    hover_name='Ticker',
    hover_data={
        'price': ':.2f',      # Show actual price in hover
        'log_price': False,   # Hide log price from hover
        'volume': ':,.0f',    # Show formatted volume
        'size': False,        # Hide size from hover
        'date': False         # Hide date from hover (we have date_str)
    },
    animation_frame='date_str',  # each day is a frame
    animation_group='Ticker',
    size_max=60,
    range_y=[df['log_price'].min()*0.98, df['log_price'].max()*1.02],
    range_x=[df['date'].min(), df['date'].max()],
    title='BIST Stock Prices Motion Chart - Log Scale (Circle Size = Volume)',
    labels={
        'log_price': 'Log Price',
        'date': 'Date',
        'Ticker': 'Stock',
        'price': 'Price (TRY)'
    }
)

# Customize layout for better log scale visualization
fig.update_layout(
    yaxis_title='Log Price (TRY)',
    xaxis_title='Date',
    hovermode='closest',
    showlegend=True,
    width=1000,
    height=600
)

# Customize hover template to show meaningful information
fig.update_traces(
    hovertemplate='<b>%{hovertext}</b><br>' +
                  'Date: %{customdata[0]}<br>' +
                  'Price: ₺%{customdata[1]:.2f}<br>' +
                  'Volume: %{customdata[2]:,.0f}<br>' +
                  '<extra></extra>'
)

# Update axes
fig.update_xaxes(type='date', showgrid=True)
fig.update_yaxes(showgrid=True)

# Add play button
fig.update_layout(
    updatemenus=[{
        'type': 'buttons',
        'direction': 'left',
        'showactive': False,
        'x': 0.1,
        'y': 0,
        'xanchor': 'right',
        'yanchor': 'top',
        'bgcolor': 'rgba(255, 255, 255, 0.8)',
        'bordercolor': 'lightgray',
        'borderwidth': 1,
        'buttons': [
            {
                'label': '▶️ Play',
                'method': 'animate',
                'args': [
                    None,
                    {
                        'frame': {'duration': 100, 'redraw': True},
                        'fromcurrent': True,
                        'transition': {'duration': 50, 'easing': 'linear'}
                    }
                ]
            },
            {
                'label': '⏸️ Pause',
                'method': 'animate',
                'args': [
                    [None],
                    {
                        'frame': {'duration': 0, 'redraw': True},
                        'mode': 'immediate',
                        'transition': {'duration': 0}
                    }
                ]
            },
            {
                'label': '⏹️ Stop',
                'method': 'animate',
                'args': [
                    [None],
                    {
                        'frame': {'duration': 0, 'redraw': True},
                        'mode': 'immediate',
                        'transition': {'duration': 0}
                    }
                ]
            }
        ]
    }]
)

fig.show()

/tmp/ipython-input-3997714911.py:10: FutureWarning:

YF.download() has changed argument auto_adjust default to True

/tmp/ipython-input-3997714911.py:13: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.

